## brand

---

In [1]:
import os
import pandas as pd
import requests
import json

from auth import NotiServerAuth
from dataclasses import dataclass, field
from typing import Optional, List
from urllib.parse import urlparse, parse_qs

from pprint import pprint

In [2]:
class NotiServerRequest:
    DOMAIN = 'https://hook.crwlnoti.shop'
    URL = '/plist'

    @classmethod
    def product_info(cls, *, ids: List[str], pivs: List[str], prices:bool=False) -> requests.Response:
        REQUEST_METHOD = 'GET'
        url = f"{cls.DOMAIN}{cls.URL}"
        return requests.request(
            method=REQUEST_METHOD, 
            url=url,
            headers={
                **NotiServerAuth.auth_string(REQUEST_METHOD, url),
                
            },
            params={'id': ids, 'piv': pivs, 'prices': 'Y' if prices else 'N'}
        )
    
    @classmethod
    def bulk_product_info(cls, *, ids: List[str], pivs: List[str], prices:bool=False) -> requests.Response:
        REQUEST_METHOD = 'POST'
        url = f"{cls.DOMAIN}{cls.URL}"
        return requests.request(
            method=REQUEST_METHOD, 
            url=url,
            headers={
                **NotiServerAuth.auth_string(REQUEST_METHOD, url),
                
            },
            data=json.dumps({'id': ids, 'piv': pivs, 'prices': prices})
        )


In [3]:
df = pd.read_csv('../df_interaction_unique.csv')
df_test = df.head(20)

In [4]:
# 주어진 데이터프레임에서 아이템 ID를 추출
item_ids = df_test['item'].tolist()

# 각 아이템에 대한 정보를 저장할 빈 리스트를 생성합니다.
brands = []

# 각 아이템에 대한 정보를 조회합니다.
# 브랜드 정보 조회후 브랜드 데이터가 조회되지 않는다면, title에서 brand를 가져옴
for item_id in item_ids:
    res = NotiServerRequest.product_info(ids=[], pivs=[item_id]) # ids
    try :
        brand = res.json().get('data', {}).get('products', [])[0]['details']['brand']
    except:
        brand = res.json().get('data', {}).get('products', [])[0]['title'].split()[0]
    
    brands.append(brand)

# 브랜드 정보를 데이터프레임에 새로운 열로 추가합니다.
df_test['brand'] = brands
df_test

/var/folders/8r/strv9f_x6b3ghnb0chcl145m0000gn/T/ipykernel_25471/2191805686.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['brand'] = brands


,user,item,time,brand
0,e44ba8873d8672b0b427a482996a2de0,6543958499-14581721807-81823794308,1707663694,홈플래닛
1,2be5019e06b731490ad9ff897871c4f4,7525739245-18528573610-85667036705,1707663697,MSI
2,ef1b51712d640b10f81e1f6a8ac933ea,6718137289-15618562334-82836343330,1707663751,레노버
3,f2258782edbcf31d906f092d8c891c74,7319766659-18771327394-85902976089,1707663752,로지텍
4,f2258782edbcf31d906f092d8c891c74,7849554293-21391795606-77230786697,1707663772,키크론
5,03fe7871128b0dc1d8a452927db79a0f,6466357331-14095095193-82065596766,1707663784,레노버
6,b43b2921cc01b58cdaae86f807a508cf,7579231557-20004029414-87101084113,1707663816,보스
7,f6c4bbed3e4ae410305ed1000cd764b9,6224551243-12475399161-79982556558,1707663873,에이수스
8,1a507722a32cd8b524b5cd6df55baa22,6694780697-15484383106-82703796924,1707664001,레이저
9,1a507722a32cd8b524b5cd6df55baa22,6979133373-20620449297-87694702089,1707664008,레이저


---

## cate

- 키워드 기반 분류
    - 특정 키워드나 패턴이 포함된 제품명을 해당 카테고리에 할당합니다. 
    - 예를 들어, "TV"나 "UHD"가 포함되어 있으면 "TV" 카테고리로, "노트북"이나 "코어"가 포함되어 있으면 "노트북" 카테고리로 분류할 수 있습니다.
    - 키워드 리스트 생성 : 제품명에서 빈도수가 많이 나오는 애들 힌트를 얻어서 처리

- 다중 클래스 분류 문제
    - train : item name , item cate
    - pred  : item name , ???



In [5]:
cate = [
    "노트북",     # laptop
    "태블릿",     # tablet
    "스마트폰",   # smartphone
    "스마트워치", # smartwatch
    "모니터",     # monitor
    "TV",         # tv
    "스피커",     # speaker
    "헤드폰",     # headphone
    "이어폰",     # earphone
    "게이밍",      # gaming
    "데스크탑",    # desktop
    "키보드",     # keyboard
    "마우스",     # mouse
    "로봇청소기",  # robovac
    "냉장고",     # fridge
    "세탁기",     # laundry
    "킥보드",     # scooter
    "저장장치",   # storage
    "청소기",     # cleaner
    "가전/디지털" # digital
]


In [6]:
# 주어진 데이터프레임에서 아이템 ID를 추출
item_ids = df_test['item'].tolist()

# 각 아이템에 대한 정보를 저장할 빈 리스트를 생성합니다.
cates = []

# 각 아이템에 대한 정보를 조회합니다.
for item_id in item_ids:
    try:
        res = NotiServerRequest.product_info(ids=[], pivs=[item_id])
        # 'details'에 'brand' 정보가 있는지 확인하고, 없는 경우 'title' 정보로 대체합니다.
        cate = res.json().get('data', {}).get('products', [])[0]['details']['category_small']
        # print(cate)
        cates.append(cate)
    
    except KeyError:
        cate = res.json().get('data', {}).get('products', [])[0]['title'].split(",")[0]
        # print(cate)
        cates.append(cate)

        
# 브랜드 정보를 데이터프레임에 새로운 열로 추가합니다.
df_test['cate'] = cates
df_test

/var/folders/8r/strv9f_x6b3ghnb0chcl145m0000gn/T/ipykernel_25471/198858624.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['cate'] = cates


,user,item,time,brand,cate
0,e44ba8873d8672b0b427a482996a2de0,6543958499-14581721807-81823794308,1707663694,홈플래닛,TV
1,2be5019e06b731490ad9ff897871c4f4,7525739245-18528573610-85667036705,1707663697,MSI,MSI 2023 스워드 GF66 15.6
2,ef1b51712d640b10f81e1f6a8ac933ea,6718137289-15618562334-82836343330,1707663751,레노버,노트북
3,f2258782edbcf31d906f092d8c891c74,7319766659-18771327394-85902976089,1707663752,로지텍,마우스
4,f2258782edbcf31d906f092d8c891c74,7849554293-21391795606-77230786697,1707663772,키크론,키보드
5,03fe7871128b0dc1d8a452927db79a0f,6466357331-14095095193-82065596766,1707663784,레노버,노트북
6,b43b2921cc01b58cdaae86f807a508cf,7579231557-20004029414-87101084113,1707663816,보스,헤드폰/헤드셋
7,f6c4bbed3e4ae410305ed1000cd764b9,6224551243-12475399161-79982556558,1707663873,에이수스,노트북
8,1a507722a32cd8b524b5cd6df55baa22,6694780697-15484383106-82703796924,1707664001,레이저,PC스피커/헤드셋/기타
9,1a507722a32cd8b524b5cd6df55baa22,6979133373-20620449297-87694702089,1707664008,레이저,노트북


In [7]:
# 카테고리별 키워드 정의
category_keywords = {
    '노트북': ['노트북','맥북', '그램','ThinkPad','스트릭스','스워드','Zenbook', '블레이드','에일리언웨어', '갤럭시북', '아이디어패드','울트라PC','코어i7', '코어i5', '코어i9','라이젠'],
    '스마트폰': [],
    '헤드폰': ['헤드폰','헤드셋'],
    '스피커': ['스피커'],
    '냉장고': ['냉장고'],
    '마우스': ['마우스', 'G PRO'],
    '청소기': ['청소기'],
    '키보드': ['키보드','키크론'],
    'TV': ['TV','HD']
}

# 분류를 위한 함수 정의
def categorize_by_keywords(item, category_keywords):
    for category, keywords in category_keywords.items():
        if any(keyword in item for keyword in keywords):
            return category
    return item  # 키워드가 일치하지 않는 경우 원래 카테고리를 반환

# 'cate' 열에 함수 적용
df_test['cate'] = df_test['cate'].apply(lambda x: categorize_by_keywords(x, category_keywords))
df_test

/var/folders/8r/strv9f_x6b3ghnb0chcl145m0000gn/T/ipykernel_25471/1454681606.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['cate'] = df_test['cate'].apply(lambda x: categorize_by_keywords(x, category_keywords))


,user,item,time,brand,cate
0,e44ba8873d8672b0b427a482996a2de0,6543958499-14581721807-81823794308,1707663694,홈플래닛,TV
1,2be5019e06b731490ad9ff897871c4f4,7525739245-18528573610-85667036705,1707663697,MSI,노트북
2,ef1b51712d640b10f81e1f6a8ac933ea,6718137289-15618562334-82836343330,1707663751,레노버,노트북
3,f2258782edbcf31d906f092d8c891c74,7319766659-18771327394-85902976089,1707663752,로지텍,마우스
4,f2258782edbcf31d906f092d8c891c74,7849554293-21391795606-77230786697,1707663772,키크론,키보드
5,03fe7871128b0dc1d8a452927db79a0f,6466357331-14095095193-82065596766,1707663784,레노버,노트북
6,b43b2921cc01b58cdaae86f807a508cf,7579231557-20004029414-87101084113,1707663816,보스,헤드폰
7,f6c4bbed3e4ae410305ed1000cd764b9,6224551243-12475399161-79982556558,1707663873,에이수스,노트북
8,1a507722a32cd8b524b5cd6df55baa22,6694780697-15484383106-82703796924,1707664001,레이저,헤드폰
9,1a507722a32cd8b524b5cd6df55baa22,6979133373-20620449297-87694702089,1707664008,레이저,노트북


---